# Deep Neural Networks Inference Using FHE

## Introduction
This notebook demonstrates running inference of deep neural networks under FHE, where the neural networks demonstrated are AlexNet, SqueezeNet and ResNet-18.

AlexNet is the name of a convolutional neural network which has had a large impact on the field of machine learning, specifically in the application of deep learning to machine vision. This is a complicated data science use case for image classification using an FHE-encoded AlexNet neural network with 5 convolutional layers and 3 fully connected layers. As you can see, there is not a lot of code required to perform this under FHE.

SqueezeNet and ResNet-18 are additional convolutional neural networks with 26 and 20 convolutional layers, respectively, organized as a non-sequential graph.

#### Note: running this demo requires up to ~80GB, ~100GB and ~150GB of available RAM for AlexNet, SqueezeNet and ResNet-18, respectively.

#### Note: the prediction part of this demo is expected to last ~3:30, ~8:45 and ~16:00 minutes for AlexNet, SqueezeNet and ResNet-18, respectively, with 88 CPUs.

## Use case
A potential use case in the healthcare domain using the AlextNet demo is encrypted disease detection or genetic risk prediction using a cloud service. A hospital’s data center is unlikely to match the scalability and efficiency of a cloud service, particularly in hosting workloads for large clinical trials for complex genetic diseases. However, due to privacy risks and healthcare regulations, it is often impractical for hospitals to make the transition to cloud. FHE in clinical research can improve the acceptance of data-sharing protocols, increase sample sizes and accelerate learning from real-world data. Encrypted image classification can be performed in the cloud service environment to determine if a patient has a specific health issue or disease given an encrypted data set sent by the hospital.

<br>

References:

<sub><sup> Krizhevsky, Alex and Sutskever, Ilya and Hinton, Geoffrey E. "Imagenet classification with deep convolutional neural networks", Advances in neural information processing systems 25, 2012. </sup></sub>

<br>

## Step 1. Unencrypted deep neural network model

#### 1a. Import and initialize

In [ ]:
import numpy as np
import os
import psutil
import utils
import pyhelayers

input_dir = utils.get_data_sets_dir() + '/' # for the model architecture file

model_architecture = 'AlexNet'
# model_architecture = 'SqueezeNet'
# model_architecture = 'ResNet-18'

<br>

#### 1b. Load the network model with random weights

In [ ]:
hyper_params = pyhelayers.PlainModelHyperParams()
hyper_params.init_random_weights = True
hyper_params.min_rand_value = -0.1
hyper_params.max_rand_value = 0.1
hyper_params.sparse_rate = 0.5
hyper_params.verbose = True

nnp = pyhelayers.NeuralNetPlain()

if model_architecture == 'AlexNet':
    utils.verify_memory(min_memory_size=90)
    nnp.init_from_files(hyper_params, [input_dir + "net_alex/model_same_padding.json"])
elif model_architecture == 'SqueezeNet':
    utils.verify_memory(min_memory_size=120)
    nnp.init_from_files(hyper_params, [input_dir + "squeeze_net/model.onnx"])
elif model_architecture == 'ResNet-18':
    utils.verify_memory(min_memory_size=180)
    nnp.init_from_files(hyper_params, [input_dir + "res_net_18/model.onnx"])

print("Loaded plain model")

<br>

#### 1c. Compilation (automatic optimizations)

In [ ]:
he_run_req = pyhelayers.HeRunRequirements()
# Request a HEaaN context if available, or a SEAL context otherwise
he_run_req.set_he_context_options([pyhelayers.HeContext.create(["HEaaN_CKKS", "SEAL_CKKS"])])
he_run_req.set_model_encrypted(False)
he_run_req.set_lazy_encoding(True)

profile = pyhelayers.HeModel.compile(nnp, he_run_req)
batch_size = profile.get_optimal_batch_size()
print('Profile ready. Batch size=',batch_size)

<br>

#### 1d. Generate random test data

In [ ]:
plain_input_shape = nnp.get_input_shapes_for_predict()[0]
plain_input_shape[0] = batch_size

plain_samples = np.random.uniform(-0.5, 0.5, plain_input_shape)
print("Generated random plain samples")

<br>

#### 1e. Initialize context

In [ ]:
%%time

context = pyhelayers.HeModel.create_context(profile)
print('HE Context ready')

process = psutil.Process(os.getpid())
print("Memory consumption (GB): ", process.memory_info().rss / (1000*1000*1000))

<br>

## Part 2. Encoded deep Neural Network Model

#### 2a. Encode the model using FHE

In [ ]:
%%time

nn = pyhelayers.NeuralNet(context)
nn.encode(nnp, profile)
print('HE model ready')

process = psutil.Process(os.getpid())
print("Memory consumption (GB): ", process.memory_info().rss / (1000*1000*1000))

<br>

#### 2b. Encrypt the input samples with the random image created earlier

In [ ]:
%%time

iop = nn.create_io_processor()
samples = pyhelayers.EncryptedData(context)
iop.encode_encrypt_inputs_for_predict(samples, [plain_samples])
print('Encrypted samples ready')

process = psutil.Process(os.getpid())
print("Memory consumption (GB): ", process.memory_info().rss / (1000*1000*1000))

<br>

#### 2c. Perform inference under encryption 


In [ ]:
%%time

predictions = pyhelayers.EncryptedData(context)
utils.start_timer()
nn.predict(predictions, samples)
duration=utils.end_timer('predict')

print('Prediction done')
utils.report_duration('predict per sample',duration/batch_size)

process = psutil.Process(os.getpid())
print("Memory consumption (GB): ", process.memory_info().rss / (1000*1000*1000))

<br>

#### 2d. Decrypt results

In [ ]:
%%time

plain_predictions = iop.decrypt_decode_output(predictions)
print('Decrypted predictions ready')

process = psutil.Process(os.getpid())
print("Memory consumption (GB): ", process.memory_info().rss / (1000*1000*1000))

In [ ]:
print(plain_predictions)